<a href="https://colab.research.google.com/github/Djensonsan/IR-Assignment-2/blob/main/LDA-Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 2: LDA


## Imports

In [1]:
# Install your required packages here
!pip install pandas numpy matplotlib fsspec gcsfs 
!pip install -q tqdm

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Preprocessing

In [ ]:
# Pycharm:
data = pd.read_csv('data/news_dataset.csv')

In [ ]:
# Colab:
data = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/news_dataset.csv')

### Exploration

In [ ]:
data.info()

In [ ]:
data.head(n=3)

## LDA Implementation

## Topic Generation

## Experimentation

## Evaluation

## Top Documents